In [38]:
# Import dependencies
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

In [39]:
# load the data
df = pd.read_csv("Resources/class_data.csv")
df

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,...,Lat,Long,Per Capita Income,Currency Conv to USD,Total_Dependents,Total_campaigns,age,edu_classes,relation_status,class
0,0,1826,1970,Graduation,Divorced,84835.0,0,0,2014-06-16,0,...,40.416775,-3.70379,"27,057.2",0.846231,0,0,44,2,4,0
1,1,5371,1989,Graduation,Single,21474.0,1,0,2014-04-08,0,...,40.416775,-3.70379,"27,057.2",0.846231,1,1,25,2,1,1
2,2,7348,1958,PhD,Single,71691.0,0,0,2014-03-17,0,...,40.416775,-3.70379,"27,057.2",0.846231,0,0,56,5,1,0
3,3,1991,1967,Graduation,Together,44931.0,0,1,2014-01-18,0,...,40.416775,-3.70379,"27,057.2",0.846231,1,0,47,2,2,1
4,4,5642,1979,Master,Together,62499.0,1,0,2013-12-09,0,...,40.416775,-3.70379,"27,057.2",0.846231,1,0,35,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,2177,9940,1958,Graduation,Together,64961.0,0,1,2012-12-23,97,...,-25.731340,28.21837,"5,090.7",14.717459,1,0,56,2,2,0
2178,2178,3406,1964,Graduation,Single,45989.0,0,1,2012-10-22,97,...,-25.731340,28.21837,"5,090.7",14.717459,1,0,50,2,1,0
2179,2179,313,1968,Graduation,Widow,73455.0,0,0,2013-10-28,98,...,-25.731340,28.21837,"5,090.7",14.717459,0,1,46,2,5,2
2180,2180,5871,1979,Master,Together,24401.0,0,0,2012-08-31,98,...,-25.731340,28.21837,"5,090.7",14.717459,0,0,35,4,2,1


In [40]:
# Add the count of campaign 6 to Total_campaigns
df['Total_campaigns'] = df['Total_campaigns'] + df['Response']

In [41]:
# Filter only the rows for Class 2
class_2_df = df.loc[df['class'] == 2]
class_2_df.head()

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,...,Lat,Long,Per Capita Income,Currency Conv to USD,Total_Dependents,Total_campaigns,age,edu_classes,relation_status,class
15,15,7962,1987,PhD,Single,95169.0,0,0,2013-10-09,1,...,40.416775,-3.70379,"27,057.2",0.846231,0,3,27,5,1,2
17,17,3725,1961,PhD,Single,84865.0,0,0,2013-05-09,1,...,40.416775,-3.70379,"27,057.2",0.846231,0,5,53,5,1,2
19,19,7030,1955,PhD,Married,66465.0,0,1,2013-03-30,1,...,40.416775,-3.70379,"27,057.2",0.846231,1,1,59,5,3,2
54,54,6815,1980,2n Cycle,Married,96547.0,0,0,2014-05-23,4,...,40.416775,-3.70379,"27,057.2",0.846231,0,4,34,3,3,2
60,60,1945,1962,Graduation,Married,71434.0,0,1,2013-09-18,4,...,40.416775,-3.70379,"27,057.2",0.846231,1,2,52,2,3,2


In [42]:
# create a dataframe for country value counts
country_value_count = pd.DataFrame(class_2_df.Country.value_counts())

# create a dataframe a blank cmp_country_df with 2 columns
cmp_country_df = pd.DataFrame(columns=['country','total_customers'])

# assign the list of coutries in the country_value_count dataframe to the 'country' column of the cmp_country_df dataframe
cmp_country_df['country'] = country_value_count.index.to_list()

# assign the value counts in the country_value_count dataframe to the 'total_customers' column of the cmp_country_df dataframe
cmp_country_df['total_customers'] = country_value_count.Country.to_list()
cmp_country_df

,country,total_customers
0,SP,129
1,SA,38
2,CA,30
3,AUS,16
4,GER,15
5,IND,14
6,US,9


In [43]:
# Retrieve the number of responders to each campaign for each country
cmps = ['AcceptedCmp1','AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5','Response']
sp = class_2_df[class_2_df.Country == 'SP'][cmps].sum().to_list()
sa = class_2_df[class_2_df.Country == 'SA'][cmps].sum().to_list()
ca = class_2_df[class_2_df.Country == 'CA'][cmps].sum().to_list()
aus = class_2_df[class_2_df.Country == 'AUS'][cmps].sum().to_list()
ger = class_2_df[class_2_df.Country == 'GER'][cmps].sum().to_list()
ind = class_2_df[class_2_df.Country == 'IND'][cmps].sum().to_list()
us = class_2_df[class_2_df.Country == 'US'][cmps].sum().to_list()

In [44]:
# append the above values to the cmp_country_df dataframe
cmp_country_df[['cmp1_response','cmp2_response', 'cmp3_response', 'cmp4_response', 'cmp5_response','cmp6_response']] = [sp,sa,ca,aus,ger,ind,us]
cmp_country_df

,country,total_customers,cmp1_response,cmp2_response,cmp3_response,cmp4_response,cmp5_response,cmp6_response
0,SP,129,65,15,25,49,84,60
1,SA,38,18,3,5,12,21,18
2,CA,30,17,4,5,14,20,18
3,AUS,16,6,0,4,5,12,9
4,GER,15,7,2,2,6,8,7
5,IND,14,7,2,3,5,6,5
6,US,9,6,0,1,1,5,3


In [45]:
# Retrieve the number of customers who responded to at least 1 campaign for each country
c1=class_2_df[class_2_df.Country == 'SP'][class_2_df.Total_campaigns > 0]['ID'].count()
c2=class_2_df[class_2_df.Country == 'SA'][class_2_df.Total_campaigns > 0]['ID'].count()
c3=class_2_df[class_2_df.Country == 'CA'][class_2_df.Total_campaigns > 0]['ID'].count()
c4=class_2_df[class_2_df.Country == 'AUS'][class_2_df.Total_campaigns > 0]['ID'].count()
c5=class_2_df[class_2_df.Country == 'GER'][class_2_df.Total_campaigns > 0]['ID'].count()
c6=class_2_df[class_2_df.Country == 'IND'][class_2_df.Total_campaigns > 0]['ID'].count()
c7=class_2_df[class_2_df.Country == 'US'][class_2_df.Total_campaigns > 0]['ID'].count()

C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\HP\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: 

In [46]:
# add a column called 'cmps_response' whose values are the number of customers who responded to at least 1 campaign
cmp_country_df['cmps_response'] = [c1,c2,c3,c4,c5,c6,c7]
cmp_country_df

,country,total_customers,cmp1_response,cmp2_response,cmp3_response,cmp4_response,cmp5_response,cmp6_response,cmps_response
0,SP,129,65,15,25,49,84,60,129
1,SA,38,18,3,5,12,21,18,37
2,CA,30,17,4,5,14,20,18,30
3,AUS,16,6,0,4,5,12,9,16
4,GER,15,7,2,2,6,8,7,15
5,IND,14,7,2,3,5,6,5,13
6,US,9,6,0,1,1,5,3,8


In [47]:
# Export the cmp_country_df dataframe into a CSV file.
cmp_country_df.to_csv("Resources/class2_cmp_country.csv", index=False)